In [1]:
# this file collects the main result to be shown in the paper.

# basically, FF vs recurrent (inst-avg) vs recurrent (inst-last) with similar numbers of parameters

In [2]:
from os.path import join, dirname, exists
from os import makedirs

from thesis_v2 import dir_dict
from thesis_v2.submission import utils
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200801_2_generator,
    keygen
)

from thesis_v2.training_extra.misc import count_params
from thesis_v2.training.training_aux import load_training_results

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import load_modules

load_modules()

import json
from copy import deepcopy
import pandas as pd

In [3]:
from torchnetjson.builder import build_net
import numpy as np

In [4]:
# get cc_max
from strflab.stats import cc_max
from thesis_v2.data.prepared.yuanyuan_8k import get_neural_data_per_trial

cc_max_all_neurons = cc_max(get_neural_data_per_trial(('042318', '043018','051018',)))
assert cc_max_all_neurons.shape == (79,)

In [5]:
def collect_main_result():
    # this does not collect non native results. those will be used in deeper analysis but not in main result.
    
    fixed_keys = {
        'kernel_size_l23': 3,
        'kernel_size_l1': 9,
        'ff_1st_block': True,
        'pooling_ksize': 3,
        'pooling_type': 'avg',
        'bn_after_fc': False,
        'rcnn_bl_psize': 1,
        'rcnn_bl_ptype': None,
        'input_size': 50,
        'split_seed': 'legacy',
        'dataset_prefix': 'yuanyuan_8k_a_3day',
        'model_prefix': 'maskcnn_polished_with_rcnn_k_bl',
        'scale_name': '0.01',
        'scale': '0.01',
        'smoothness_name': '0.000005',
        'smoothness': '0.000005',
    }
    
    rows_all = []
    
    for idx, (src, param) in enumerate(explored_models_20200801_2_generator(with_source=True)):
        assert len(param) == 26
        total_param_to_explain = len(param)
        
        if idx % 100 == 0:
            print(idx)
            
        # some parameters that won't change.
        for k_fix, v_fix in fixed_keys.items():
            assert param[k_fix] == v_fix
            total_param_to_explain -= 1
        
        # {'yhat_reduce_pick': 'none', 'train_keep': 1280, 'model_seed': 0,
        # act_fn': 'relu', 'loss_type': 'mse', 'out_channel': 8, 'num_layer': 2,
        # 'rcnn_bl_cls': 1,
        # 'rcnn_acc_type': 'cummean', 'ff_1st_bn_before_act': True}
        
        
        # load model to get param count
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        # 10 to go.
        result = load_training_results(key, return_model=False)
        # load twice, first time to get the model.
        result = load_training_results(key, return_model=True, model=build_net(result['config_extra']['model']))
        num_param = count_params(result['model'])
        
        
        cc_native = np.asarray(result['stats_best']['stats']['test']['corr'])
        
        # replace 'yhat_reduce_pick' + 'rcnn_acc_type' with 'readout_type'
        readout_raw = param['yhat_reduce_pick'], param['rcnn_acc_type']
            
        param['readout_type'] = {
            ('none', 'instant'): 'inst-avg',
            (-1, 'last'): 'inst-last',
        }[readout_raw]
        assert src == param['readout_type']

        del param['yhat_reduce_pick']
        del param['rcnn_acc_type']
        total_param_to_explain -= 1
        
        # load eval json
        eval_json_file = join(dir_dict['analyses'], key, 'eval.json')
        try:
            with open(eval_json_file, 'rt', encoding='utf-8') as f_eval:
                eval_json = json.load(f_eval)
        except FileNotFoundError as e:
            print('missing file',idx)
            raise e
        
        cc_native_debug = np.asarray(eval_json['native'])
        
        if param['rcnn_bl_cls'] != 1:
            cc_native_debug_2 = np.asarray(eval_json[param['readout_type']][str(param['rcnn_bl_cls'])])
        else:
            cc_native_debug_2 = cc_native_debug
        
        assert cc_native_debug.shape == cc_native.shape == (79,) == cc_max_all_neurons.shape == cc_native_debug_2.shape
        
        
        assert np.allclose(cc_native, cc_native_debug, atol=1e-4)
        assert np.allclose(cc_native, cc_native_debug_2, atol=1e-4)
        
        
        if param['train_keep'] is None:
            param['train_keep'] = 5120
        assert param['train_keep'] in {1280, 2560, 5120}
        
        
        # add result
        row_this = {
            k: v for k, v in param.items() if k not in fixed_keys
        }
        row_this['num_param'] = num_param
        row_this['cc_raw_avg'] = cc_native.mean()
        row_this['cc2_raw_avg'] = (cc_native**2).mean()
        row_this['cc2_normed_avg'] = ((cc_native/cc_max_all_neurons)**2).mean()
        
        
        rows_all.append(row_this)

    
    df_this = pd.DataFrame(rows_all, columns=sorted(list(rows_all[0].keys())))
    df_this = df_this.set_index(keys=sorted([k for k in param if k not in fixed_keys]), verify_integrity=True)
    return df_this

In [6]:
f_main_result = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20200801_2', 'main_result.pkl')
if not exists(f_main_result):
    makedirs(dirname(f_main_result), exist_ok=True)
    df_main_result = collect_main_result()
    df_main_result.to_pickle(f_main_result)
    del df_main_result
df_main_result = pd.read_pickle(f_main_result)

In [7]:
df_main_result

cc2_normed_avg  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep                   
relu     True                 mse       0          2         8           5           inst-avg     1280              0.339469   
         False                mse       0          2         8           5           inst-avg     1280              0.373514   
         True                 mse       0          2         8           6           inst-avg     1280              0.354674   
         False                mse       0          2         8           6           inst-avg     1280              0.374026   
         True                 mse       0          2         8           7           inst-avg     1280              0.364295   
...                                                                                                                      ...   
softplus False                poisson   1          3         32          5           inst-avg     5120              0.670983   
         True                 poisson   1          3         32          6           inst-avg     5120              0.682216   
         False                poisson   1          3         32          6           inst-avg     5120              0.678750   
         True                 poisson   1          3         32          7           inst-avg     5120              0.684505   
         False                poisson   1          3         32          7           inst-avg     5120              0.676353   

                                                                                                              cc2_raw_avg  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep                
relu     True                 mse       0          2         8           5           inst-avg     1280           0.160028   
         False                mse       0          2         8           5           inst-avg     1280           0.175525   
         True                 mse       0          2         8           6           inst-avg     1280           0.166631   
         False                mse       0          2         8           6           inst-avg     1280           0.176095   
         True                 mse       0          2         8           7           inst-avg     1280           0.171580   
...                                                                                                                   ...   
softplus False                poisson   1          3         32          5           inst-avg     5120           0.308032   
         True                 poisson   1          3         32          6           inst-avg     5120           0.313482   
         False                poisson   1          3         32          6           inst-avg     5120           0.312071   
         True                 poisson   1          3         32          7           inst-avg     5120           0.315005   
         False                poisson   1          3         32          7           inst-avg     5120           0.310776   

                                                                                                              cc_raw_avg  \
act_fn   ff_1st_bn_before_act loss_type model_seed num_layer out_channel rcnn_bl_cls readout_type train_keep               
relu     True                 mse       0          2         8           5           inst-avg     1280          0.363345   
         False                mse       0          2         8           5           inst-avg     1280          0.387667   
         True                 mse       0          2         8           6           inst-avg     1280          0.377675   
         False                mse       0          2         8           6           inst-avg     1280          0.387864   
         True                 mse       0          2         8           7 